# Emparejar metadatos

Busco fechas, estados, localidades y el identificador interno del boletín en el contenido principal del boletín.

In [1]:
from pymongo import MongoClient, UpdateOne
import re
from pprint import pprint

In [2]:
uri = open('mongo_uri.txt', 'r', encoding='utf-8')
client = MongoClient(uri)
pprint(client)
db = client.boletines_db
pprint(db.boletines.find_one())

MongoClient(host=['clustershido-shard-00-02-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-01-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-00-uxkjr.gcp.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='ClusterShido-shard-0', ssl=True)
{'_id': ObjectId('5ec3089dd380e649eb619ce9'),
 'boletin': 'Juntos haremos historia\n'
            'BOLETÍN CAMPAÑA-009\n'
            'PIDE LÓPEZ OBRADOR A TRUMP QUE NO MALTRATE A LOS MEXICANOS, '
            '“QUEREMOS RESPETO MUTUO”\n'
            '- NI LAS AMENAZAS, NI LOS MILITARES A LA FRONTERA EVITARAN QUE '
            'MÉXICO QUIERA TENER UNA RELACIÓN DE AMISTAD CON EU: AMLO\n'
            'MATAMOROS, TAMAULIPAS, 06 DE ABRIL DE 2018.- Andrés Manuel López '
            'Obrador manifestó que México y Estados Unidos “somos pueblos que '
            'estamos hermanados, los gobiernos no tienen por qué separar a los '
            'pueblos, nosotros le p

## Buscar localidad y estado

In [47]:
CANDIDATOS = ['JAMK', 'RAC']

In [60]:
patron = {}
for candidato in CANDIDATOS:
    print(f'--------------- {candidato} ---------------------')
    n_matched = 0
    n_total = 0
    
    cursor = db.boletines.find(
    {'candidato':candidato}, 
        {
            'estado':1,
            'localidad':1,
            'boletin':1,
        }
    ).sort('fecha')
    for boletin in cursor:
        n_total += 1
        b_split = boletin['boletin'].split('\n')
        try:
            print(f"{boletin['localidad']}, {boletin['estado']}")
            patron['localidad'] = re.compile(
                f".*{boletin['localidad']}.*", 
                re.IGNORECASE
            )
        except KeyError:
            print(f"{boletin['estado']}")
        patron['estado'] = re.compile(
            f".*{boletin['estado']}.*"
            if boletin['estado'] is not 'CDMX' 
            else '.*Ciudad de México.*', 
            re.IGNORECASE
        )
        for i in range(3):
            hay_match = False
            test_str = b_split[i]
            c_match = comun.match(test_str)
            match_est = patron['estado'].match(test_str)
            if match_est:
                hay_match = True
                # print(match_est)
                print('Encontré ', boletin['estado'])
            try:
                match_loc = patron['localidad'].match(test_str)
                if match_loc:
                    hay_match = True
                    # print(match_loc)
                    print('Encontré ', boletin['localidad'])
            except KeyError:
                pass
            if hay_match:
                print('Match en: ', test_str)
                n_matched += 1
                break
        if not hay_match:
            print('No encontró resultados')
        print()
    print('--- Total: ', n_total)
    print('--- Encontrados:', n_matched)
    print('\n')

--------------- JAMK ---------------------
CDMX, CDMX
No encontró resultados

Mérida, Yucatán
Encontré  Yucatán
Encontré  Mérida
Match en:  MÉRIDA, YUCATÁN, 31 DE MARZO DE 2018

Mérida, Yucatán
Encontré  Yucatán
Encontré  Mérida
Match en:  MÉRIDA, YUCATÁN, 1 DE ABRIL DE 2018

CDMX
No encontró resultados

Mérida, Yucatán
Encontré  Yucatán
Encontré  Mérida
Match en:  MÉRIDA, YUCATÁN, 1 DE ABRIL DE 2018

Hermosillo, Sonora
Encontré  Sonora
Encontré  Hermosillo
Match en:  HERMOSILLO, SONORA, 2 DE ABRIL DE 2018

Hermosillo, Sonora
Encontré  Sonora
Encontré  Hermosillo
Match en:  HERMOSILLO, SONORA, 2 DE ABRIL DE 2018

CDMX
No encontró resultados

Culiacán, Sinaloa
Encontré  Sinaloa
Encontré  Culiacán
Match en:  CULIACÁN, SINALOA, 3 DE ABRIL DE 2018

San Felipe del Progreso, Estado de México
Encontré  Estado de México
Encontré  San Felipe del Progreso
Match en:  San Felipe del Progreso, Estado de México, 3 de abril de 2018

Tizayuca, Hidalgo
Encontré  Hidalgo
Encontré  Tizayuca
Match en:  TI

CDMX, CDMX
No encontró resultados

CDMX, CDMX
No encontró resultados

CDMX, CDMX
No encontró resultados

Ciudad Victoria, Tamaulipas
Encontré  Tamaulipas
Encontré  Ciudad Victoria
Match en:  CIUDAD VICTORIA, TAMAULIPAS, 30 DE MAYO DE 2018

Tepic, Nayarit
Encontré  Nayarit
Encontré  Tepic
Match en:  TEPIC, NAYARIT, 31 DE MAYO DE 2018

Monterrey, Nuevo León
Encontré  Nuevo León
Encontré  Monterrey
Match en:  MONTERREY, NUEVO LEÓN, 31 DE MAYO DE 2018

Ecatepec, Estado de México
Encontré  Estado de México
Encontré  Ecatepec
Match en:  ECATEPEC, ESTADO DE MÉXICO, 1 DE JUNIO DE 2018

Puerto Vallarta, Jalisco
Encontré  Jalisco
Encontré  Puerto Vallarta
Match en:  PUERTO VALLARTA, JALISCO, 1 DE JUNIO DE 2018

Colima, Colima
Encontré  Colima
Encontré  Colima
Match en:  COLIMA, COLIMA, 2 DE JUNIO DE 2018

Pachuca de Soto, Hidalgo
Encontré  Hidalgo
Encontré  Pachuca de Soto
Match en:  PACHUCA DE SOTO, HIDALGO, 3 DE JUNIO DE 2018

San Luis Potosí, San Luis Potosí
Encontré  San Luis Potosí
Encontré

## AMLO

In [19]:
cursor = db.boletines.find(
    {'candidato':'AMLO'}, 
    {
        'estado':1,
        'localidad':1,
        'boletin':1,
    }
)

In [23]:
patron = re.compile(r'(.+)\.- ')
n_matched = 0
n_total = 0
for boletin in cursor:
    n_total += 1
    match = patron.search(boletin['boletin'])
    id_interno = boletin['boletin'].split('\n')[1]
    print(id_interno)
    if match:
        n_matched += 1
        print(match[0])
        loc_est_fecha = match[0][:-3]#.split(', ')
        print(loc_est_fecha)
        try:
            print(f"{boletin['localidad']}, {boletin['estado']}")
            patron_loc = re.compile(
                re.escape(boletin['localidad']), 
                re.IGNORECASE
            )
            match_loc = patron_loc.match(loc_est_fecha)
            if match_loc:
                print(match_loc)
        except KeyError:
            print(f"{boletin['estado']}")
#         print(f'El patrón termina en pos {match.endpos}')
        
        patron_est = re.compile(
            re.escape(boletin['estado']), 
            re.IGNORECASE
        )
        match_est = patron_est.match(loc_est_fecha)
        if match_est:
            print(match_est)     
        print()
    else:
        print('No hay patrón en', fname)
cursor.rewind()

BOLETÍN CAMPAÑA-009
MATAMOROS, TAMAULIPAS, 06 DE ABRIL DE 2018.- 
MATAMOROS, TAMAULIPAS, 06 DE ABRIL DE 2018
Matamoros, Tamaulipas
<re.Match object; span=(0, 9), match='MATAMOROS'>

BOLETÍN CAMPAÑA-004
SAN PEDRO, COAHUILA, 04 DE ABRIL DE 2018.- 
SAN PEDRO, COAHUILA, 04 DE ABRIL DE 2018
San Pedro, Coahuila
<re.Match object; span=(0, 9), match='SAN PEDRO'>

BOLETÍN CAMPAÑA-022
TIJUANA, BAJA CALIFORNIA 13 DE ABRIL DE 2018.- 
TIJUANA, BAJA CALIFORNIA 13 DE ABRIL DE 2018
Tijuana, Baja California
<re.Match object; span=(0, 7), match='TIJUANA'>

BOLETÍN CAMPAÑA-023
ROSARITO, BAJA CALIFORNIA, 15 DE ABRIL DE 2018.- 
ROSARITO, BAJA CALIFORNIA, 15 DE ABRIL DE 2018
Rosarito, Baja California
<re.Match object; span=(0, 8), match='ROSARITO'>

BOLETÍN CAMPAÑA-028
GUASAVE, SINALOA, 19 DE ABRIL DE 2018.- 
GUASAVE, SINALOA, 19 DE ABRIL DE 2018
Guasave, Sinaloa
<re.Match object; span=(0, 7), match='GUASAVE'>

BOLETÍN CAMPAÑA-031
NICOLÁS ROMERO, ESTADO DE MÉXICO 24 DE ABRIL DE 2018.- 
NICOLÁS ROMERO, ESTAD

In [24]:
print('n_matched:', n_matched)
print('n_total:', n_total)

n_matched: 84
n_total: 84


In [ ]:
for boletin in cursor:
    id_interno = boletin['boletin'].split('\n')[1]
    print(id_interno)
    cambios.append(UpdateOne(
        {
            '_id': boletin['_id']
        },
        {
            '$set': {'id_interno':id_interno}
        }
    ))
cursor.rewind()

In [25]:
write_res = db.boletines.bulk_write(cambios)
write_res.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 84,
 'nModified': 84,
 'nRemoved': 0,
 'upserted': []}

## JAMK

In [28]:
cursor = db.boletines.find(
    {'candidato':'JAMK'}, 
    {
        'estado':1,
        'localidad':1,
        'boletin':1,
    }
).sort('fecha')

In [31]:
comun = re.compile(r'COMUNICADO')
cambios = []
for boletin in cursor:
    id_interno = boletin['boletin'].split('\n')[0]
    c_match = comun.search(id_interno)
    if c_match:
        print(id_interno)
        cambios.append(UpdateOne(
            {
            '_id': boletin['_id']
            },
            {
                '$set': {'id_interno':id_interno}
            }
        ))
cursor.rewind()

COMUNICADO DE PRENSA 119
COMUNICADO DE PRENSA 121
COMUNICADO DE PRENSA 122
COMUNICADO DE PRENSA 123
COMUNICADO DE PRENSA 124
COMUNICADO DE PRENSA 126
COMUNICADO DE PRENSA 125
COMUNICADO DE PRENSA 127
COMUNICADO DE PRENSA 128
COMUNICADO DE PRENSA 006
COMUNICADO DE PRENSA 129
COMUNICADO DE PRENSA 130
COMUNICADO DE PRENSA 133
COMUNICADO DE PRENSA 132
COMUNICADO DE PRENSA 135
COMUNICADO DE PRENSA 134
COMUNICADO DE PRENSA 006
COMUNICADO DE PRENSA 136
COMUNICADO DE PRENSA 138
COMUNICADO DE PRENSA 008
COMUNICADO DE PRENSA 141
COMUNICADO DE PRENSA 139
COMUNICADO DE PRENSA 140
COMUNICADO DE PRENSA 143
COMUNICADO DE PRENSA 142
COMUNICADO DE PRENSA 144
COMUNICADO DE PRENSA 145
COMUNICADO DE PRENSA 146
COMUNICADO DE PRENSA 147
COMUNICADO DE PRENSA 148
COMUNICADO DE PRENSA 151
COMUNICADO DE PRENSA 150
COMUNICADO DE PRENSA 149
COMUNICADO DE PRENSA 154
COMUNICADO DE PRENSA 152
COMUNICADO DE PRENSA 153
COMUNICADO DE PRENSA 155
COMUNICADO DE PRENSA 156
COMUNICADO DE PRENSA 157
COMUNICADO DE PRENSA 158


In [32]:
write_res = db.boletines.bulk_write(cambios)
write_res.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 145,
 'nModified': 145,
 'nRemoved': 0,
 'upserted': []}

## RAC

In [35]:
cursor = db.boletines.find(
    {'candidato':'RAC'}, 
    {
        'estado':1,
        'localidad':1,
        'boletin':1,
    }
).sort('fecha')

In [41]:
comun = re.compile(r'COMUNICADO', re.IGNORECASE)
cambios = []
for boletin in cursor:
    b_split = boletin['boletin'].split('\n')
    for i in range(3):
        id_interno = b_split[i]
        c_match = comun.search(id_interno)
        if c_match:
            print(id_interno)
            cambios.append(UpdateOne(
                {
                '_id': boletin['_id']
                },
                {
                    '$set': {'id_interno':id_interno}
                }
            ))
            break
cursor.rewind()

COMUNICADO 01/2018
COMUNICADO 03/2018
COMUNICADO 06/2018
COMUNICADO 07/2018
COMUNICADO 08/2018
COMUNICADO 09/2018
COMUNICADO 11/2018
COMUNICADO 10/2018
COMUNICADO 12/2018
COMUNICADO 16/2018
COMUNICADO 17/2018
COMUNICADO 18/2018
COMUNICADO 20/2018
COMUNICADO 19/2018
COMUNICADO 22/2018
COMUNICADO 23/2018
COMUNICADO 24/2018
COMUNICADO 25/2018
COMUNICADO 27/2018
COMUNICADO 28/2018
COMUNICADO 29/2018
COMUNICADO 30/2018
COMUNICADO 31/2018
COMUNICADO 32/2018
COMUNICADO 33/2018
COMUNICADO 34/2018
COMUNICADO 35/2018
COMUNICADO 36/2018
COMUNICADO 38/2018
COMUNICADO 39/2018
COMUNICADO 41/2018
COMUNICADO 40/2018
COMUNICADO 42/2018
COMUNICADO 43/2018
COMUNICADO 44/2018
COMUNICADO 45/2018
COMUNICADO 46/2018
COMUNICADO 47/2018
COMUNICADO 48/2018
COMUNICADO 49/2018
COMUNICADO 50/2018
COMUNICADO 51/2018
COMUNICADO 52/2018
COMUNICADO 54/2018
COMUNICADO 53/2018
COMUNICADO 55/2018
COMUNICADO 56/2018
COMUNICADO 57/2018
COMUNICADO 58/2018
COMUNICADO 59/2018
COMUNICADO 60/2018
COMUNICADO 61/2018
COMUNICADO 6

In [42]:
write_res = db.boletines.bulk_write(cambios)
write_res.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 108,
 'nModified': 2,
 'nRemoved': 0,
 'upserted': []}